In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd

In [2]:

# تحميل البيانات
X = pd.read_csv("C:/LINCS_DATA/project/features/X_train_val.csv")
y = pd.read_csv("C:/LINCS_DATA/project/features/y_train_val.csv")['label']
X_test = pd.read_csv("C:/LINCS_DATA/project/features/X_test_unseen.csv")
y_test = pd.read_csv("C:/LINCS_DATA/project/features/y_test_unseen.csv")['label']

In [4]:
# نحذف الأعمدة النصية
X_clean = X.drop(columns=['drugA_name', 'sig_id_A', 'drugB_name', 'sig_id_B'], errors='ignore')
X_test_clean = X_test.drop(columns=['drugA_name', 'sig_id_A', 'drugB_name', 'sig_id_B'], errors='ignore')

# تطبيع البيانات
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)
X_test_scaled = scaler.transform(X_test_clean)


In [9]:
# تقسيم داخلي إلى Train و Validation (85/15)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.15, random_state=42, stratify=y
)


In [17]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# استخدم y الموجود فعلياً
classes = np.unique(y)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)

# نحوله إلى dict
class_weight_dict = {int(cls): weight for cls, weight in zip(classes, class_weights)}
print("📊 Class Weights:", class_weight_dict)


📊 Class Weights: {0: np.float64(0.6137931034482759), 1: np.float64(2.696969696969697)}


In [19]:
# بناء النموذج
model = Sequential([
    Dense(512, activation='relu', input_shape=(2001,)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

C:\Users\Reem Ramadan\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),

    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(1, activation='sigmoid')
])


C:\Users\Reem Ramadan\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
from keras.optimizers import Adam

# التجميع

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])


In [31]:
early_stop = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    class_weight=class_weight_dict,
    verbose=1
)

Epoch 1/50


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.7498 - loss: 0.5869 - val_accuracy: 0.9259 - val_loss: 0.1238
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8197 - loss: 0.2876 - val_accuracy: 0.9259 - val_loss: 0.2632
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8402 - loss: 0.3007 - val_accuracy: 0.8889 - val_loss: 0.3222
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8027 - loss: 0.3671 - val_accuracy: 0.8889 - val_loss: 0.2806
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8913 - loss: 0.2750 - val_accuracy: 0.8889 - val_loss: 0.2651
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8886 - loss: 0.1975 - val_accuracy: 0.8889 - val_loss: 0.3987
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8316 - loss: 0.2708 - val_accuracy: 0.8889 - val_loss: 0.3462
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8450 - loss: 0.3038 - val_accuracy: 0.8889 - val_loss: 0.2981
Epoch 9/50


In [ ]:
# تقييم على test
y_pred_prob = model.predict(X_test_scaled)
y_pred_class = (y_pred_prob > 0.5).astype(int)

print("📊 Evaluation on Unseen Test:")
print(classification_report(y_test, y_pred_class))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_prob))

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
📊 Evaluation on Unseen Test:
              precision    recall  f1-score   support

           0       0.71      0.94      0.80       218
           1       0.36      0.09      0.14        93

    accuracy                           0.68       311
   macro avg       0.53      0.51      0.47       311
weighted avg       0.60      0.68      0.61       311

ROC-AUC: 0.551297227976719


: 

In [15]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# التنبؤ على بيانات الاختبار
y_pred_proba = model.predict(X_test_scaled)
y_pred = (y_pred_proba > 0.5).astype(int)

# تحويل y_test إلى array لو كان DataFrame
y_true = y_test.values.ravel()

# حساب المقاييس
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_proba)
conf_matrix = confusion_matrix(y_true, y_pred)

# عرض النتائج
print("📊 Model Evaluation Metrics:")
print(f"✅ Accuracy     : {accuracy:.4f}")
print(f"✅ Precision    : {precision:.4f}")
print(f"✅ Recall       : {recall:.4f}")
print(f"✅ F1-score     : {f1:.4f}")
print(f"✅ ROC-AUC      : {roc_auc:.4f}")
print("\n🧾 Confusion Matrix:")
print(conf_matrix)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
📊 Model Evaluation Metrics:
✅ Accuracy     : 0.6913
✅ Precision    : 0.4545
✅ Recall       : 0.1613
✅ F1-score     : 0.2381
✅ ROC-AUC      : 0.5896

🧾 Confusion Matrix:
[[200  18]
 [ 78  15]]
